In [26]:
#Holistic Nested Edge Detection
#This code is only used to generate scene 5 images which are essentailly a flashback.

#This deep learning model was developed as a project in one of our undergraduate classes.
#The github repo is attached below:

# https://github.com/chethanmahindrakar/HED-and-Shape-detection

import torch
import numpy as np
from PIL import Image, ImageOps

# Ensure the PyTorch version is at least 1.3.0
assert(int(''.join(torch.__version__.split('.')[0:2])) >= 13)

# Set to not compute gradients for performance enhancement
torch.set_grad_enabled(False)
torch.backends.cudnn.enabled = False

# Model argument for the HED (Holistically-Nested Edge Detection) network
arguments_strModel = 'bsds500'

class Network(torch.nn.Module):
    """
    Defines the HED network structure.
    """
    def __init__(self):
        super(Network, self).__init__()
        
        self.netVggOne = torch.nn.Sequential(
			torch.nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1),
			torch.nn.ReLU(inplace=False),
			torch.nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1),
			torch.nn.ReLU(inplace=False)
		)

        self.netVggTwo = torch.nn.Sequential(
			torch.nn.MaxPool2d(kernel_size=2, stride=2),
			torch.nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
			torch.nn.ReLU(inplace=False),
			torch.nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1),
			torch.nn.ReLU(inplace=False)
		)

        self.netVggThr = torch.nn.Sequential(
			torch.nn.MaxPool2d(kernel_size=2, stride=2),
			torch.nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1),
			torch.nn.ReLU(inplace=False),
			torch.nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1),
			torch.nn.ReLU(inplace=False),
			torch.nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1),
			torch.nn.ReLU(inplace=False)
		)

        self.netVggFou = torch.nn.Sequential(
			torch.nn.MaxPool2d(kernel_size=2, stride=2),
			torch.nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1),
			torch.nn.ReLU(inplace=False),
			torch.nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1),
			torch.nn.ReLU(inplace=False),
			torch.nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1),
			torch.nn.ReLU(inplace=False)
		)

        self.netVggFiv = torch.nn.Sequential(
			torch.nn.MaxPool2d(kernel_size=2, stride=2),
			torch.nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1),
			torch.nn.ReLU(inplace=False),
			torch.nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1),
			torch.nn.ReLU(inplace=False),
			torch.nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1),
			torch.nn.ReLU(inplace=False)
		)

        self.netScoreOne = torch.nn.Conv2d(in_channels=64, out_channels=1, kernel_size=1, stride=1, padding=0)
        self.netScoreTwo = torch.nn.Conv2d(in_channels=128, out_channels=1, kernel_size=1, stride=1, padding=0)
        self.netScoreThr = torch.nn.Conv2d(in_channels=256, out_channels=1, kernel_size=1, stride=1, padding=0)
        self.netScoreFou = torch.nn.Conv2d(in_channels=512, out_channels=1, kernel_size=1, stride=1, padding=0)
        self.netScoreFiv = torch.nn.Conv2d(in_channels=512, out_channels=1, kernel_size=1, stride=1, padding=0)

        self.netCombine = torch.nn.Sequential(
			torch.nn.Conv2d(in_channels=5, out_channels=1, kernel_size=1, stride=1, padding=0),
			torch.nn.Sigmoid()
		)

        self.load_state_dict({ strKey.replace('module', 'net'): tenWeight for strKey, tenWeight in torch.hub.load_state_dict_from_url(url='http://content.sniklaus.com/github/pytorch-hed/network-' + arguments_strModel + '.pytorch', file_name='hed-' + arguments_strModel).items() })


    def forward(self, tenInput):
        tenBlue = (tenInput[:, 0:1, :, :] * 255.0) - 104.00698793
        
        tenGreen = (tenInput[:, 1:2, :, :] * 255.0) - 116.66876762
        tenRed = (tenInput[:, 2:3, :, :] * 255.0) - 122.67891434

        tenInput = torch.cat([ tenBlue, tenGreen, tenRed ], 1)

        tenVggOne = self.netVggOne(tenInput)
        tenVggTwo = self.netVggTwo(tenVggOne)
        tenVggThr = self.netVggThr(tenVggTwo)
        tenVggFou = self.netVggFou(tenVggThr)
        tenVggFiv = self.netVggFiv(tenVggFou)

        tenScoreOne = self.netScoreOne(tenVggOne)
        tenScoreTwo = self.netScoreTwo(tenVggTwo)
        tenScoreThr = self.netScoreThr(tenVggThr)
        tenScoreFou = self.netScoreFou(tenVggFou)
        tenScoreFiv = self.netScoreFiv(tenVggFiv)

        tenScoreOne = torch.nn.functional.interpolate(input=tenScoreOne, size=(tenInput.shape[2], tenInput.shape[3]), mode='bilinear', align_corners=False)
        tenScoreTwo = torch.nn.functional.interpolate(input=tenScoreTwo, size=(tenInput.shape[2], tenInput.shape[3]), mode='bilinear', align_corners=False)
        tenScoreThr = torch.nn.functional.interpolate(input=tenScoreThr, size=(tenInput.shape[2], tenInput.shape[3]), mode='bilinear', align_corners=False)
        tenScoreFou = torch.nn.functional.interpolate(input=tenScoreFou, size=(tenInput.shape[2], tenInput.shape[3]), mode='bilinear', align_corners=False)
        tenScoreFiv = torch.nn.functional.interpolate(input=tenScoreFiv, size=(tenInput.shape[2], tenInput.shape[3]), mode='bilinear', align_corners=False)

        return self.netCombine(torch.cat([ tenScoreOne, tenScoreTwo, tenScoreThr, tenScoreFou, tenScoreFiv ], 1))
	# end
        
netNetwork = None

def estimate(tenInput):
    """
    Process the given image tensor through the HED network to estimate the edges.
    """
    global netNetwork

    if netNetwork is None:
        netNetwork = Network().cpu().eval()

    intWidth = tenInput.shape[2]
    intHeight = tenInput.shape[1]

    return netNetwork(tenInput.cpu().view(1, 3, intHeight, intWidth))[0, :, :, :].cpu()

def process_image(input_image_path):
    """
    Given the path of an input image, this function processes the image using the HED network,
    detects edges, inverts the image colors, and returns the resulting PIL Image object.
    """
    # Convert the input image to tensor
    tenInput = torch.FloatTensor(
        np.ascontiguousarray(
            np.array(Image.open(input_image_path))[:, :, ::-1].transpose(2, 0, 1).astype(np.float32) * (1.0 / 255.0)
        )
    )

    # Estimate edges using the HED network
    tenOutput = estimate(tenInput)
    
    # Convert the tensor output to a PIL image
    output_img = Image.fromarray(
        (tenOutput.clamp(0.0, 1.0).numpy().transpose(1, 2, 0)[:, :, 0] * 255.0).astype(np.uint8)
    )
    
    # Invert colors for better visual representation of edges
    inverted_img = ImageOps.invert(output_img)
    
    return inverted_img

# Example usage:
#result_image = process_image("D:/Masters Courses/Data Science Engineering/Assignments/Assignment 4 Part 2/hed_image/3.jpg")
#result_image.save("D:/Masters Courses/Data Science Engineering/Assignments/Assignment 4 Part 2/hed_output/output.jpg")
#result_image

#print("Complete")


In [31]:
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageOps, ImageDraw, ImageFont 
import numpy as np
import pandas as pd

#Fetching the subtitles for the images
def dialogueMap(path):
    df = pd.read_csv(path, encoding='utf-8', header=None)
    df.columns = ['scene', 'text']
    return df.set_index('scene')['text'].to_dict()

#Getting the dialogue for the specific scene and specific picture in the scene
def getDialouge(lang,scene):
    return lang.get(scene)

def wrap_text(text, font, max_width, language='english'):
    """Wrap text to fit within specified width."""
    def split_english(line):
        return line.split(' ')
    def split_chinese(line):
        return list(line)
    if language == 'english':
        splitter = split_english
    elif language == 'chinese':
        splitter = split_chinese
    else:
        raise ValueError("Supported languages are 'english' and 'chinese'.")

    lines = text.split('\n')
    wrapped_lines = []

    for line in lines:
        if font.getsize(line)[0] <= max_width:
            wrapped_lines.append(line)
        else:
            # Split the line based on language
            words = splitter(line)
            new_line = words[0]
            for word in words[1:]:
                # If this word won't fit on the line, start a new line
                if font.getsize(new_line + (' ' if language == 'english' else '') + word)[0] <= max_width:
                    new_line += (' ' if language == 'english' else '') + word
                else:
                    wrapped_lines.append(new_line)
                    new_line = word
            wrapped_lines.append(new_line)

    return '\n'.join(wrapped_lines)


def add_dialogue_to_image_bottom(image_path, caption, output_path):
    TINT_COLOR = (0, 0, 0)  # Black
    TRANSPARENCY = 0.25  # Degree of transparency, 0-100%
    OPACITY = int(255 * TRANSPARENCY)
    #print(image_path)

    # Open the image
    img = Image.open(image_path).convert('RGBA')
    #img.show()
    img_width = img.width

    # Dynamic font size based on image width
    font_size = int(img_width / 15)
    if "chineese" in subs_path:
        #print("Detected chineese script")
        font = ImageFont.truetype("fonts/noto/notosans.ttf", font_size)
        max_width = img_width - 40  # 20 pixels padding on each side
        wrapped_caption = wrap_text(caption, font, max_width,'chinese')
    else:
        font = ImageFont.truetype("arial.ttf", font_size)
        max_width = img_width - 40  # 20 pixels padding on each side
        wrapped_caption = wrap_text(caption, font, max_width)

    overlay = Image.new('RGBA', img.size, TINT_COLOR + (0,))
    draw = ImageDraw.Draw(overlay)

    # Calculate the position to add text at the bottom
    text = wrapped_caption
    w, h = draw.textsize(text, font=font)
    num_lines = len(text.split('\n'))
    x, y = 20, img.height - 20 - h

    # Draw a semi-transparent background
    draw.rectangle((x, y, x + w, y + h), fill=TINT_COLOR + (OPACITY,))

    # Add text
    draw.text((x, y), text, fill=(209, 239, 8), font=font)

    # Alpha composite the two images together to obtain the desired result
    img = Image.alpha_composite(img, overlay)
    img = img.convert("RGB")  # Remove alpha for saving in jpg format

    # Save the modified image
    img.save(output_path)

#A function to brighten the images. The value controls how bright the image gets
def brighten_image(image, value=50):
    # Convert image to int16 to avoid overflow
    image_int16 = image.astype('int16')
    # Add value and clip to ensure values are between 0 and 255
    brightened_image = (image_int16 + value).clip(0, 255)
    # Convert back to uint8
    brightened_image = brightened_image.astype('uint8')
    return brightened_image

#A function to darken the images. The value controls how dark the image gets
def darken_image(image, value=50):
    # Convert image to int16 to avoid underflow
    image_int16 = image.astype('int16')
    # Subtract value and clip to ensure values are between 0 and 255
    darkened_image = (image_int16 - value).clip(0, 255)
    # Convert back to uint8
    darkened_image = darkened_image.astype('uint8')
    return darkened_image

# Add border to the image
def add_border_to_image(image, border_size= 50):
    border_color=(0, 0, 0)  #white color
    bordered_image = cv2.copyMakeBorder(image, border_size, border_size, border_size, border_size, cv2.BORDER_CONSTANT, value=border_color)
    return bordered_image

#Merging two images 
def combine_images_horizontally(image1_path, image2_path, output_path):
    # Open the images
    img1 = Image.open(image1_path)
    img2 = Image.open(image2_path)
    total_width = img1.width + img2.width
    max_height = max(img1.height, img2.height)
    new_img = Image.new('RGB', (total_width, max_height), (255, 255, 255))
    new_img.paste(img1, (0, 0))
    new_img.paste(img2, (img1.width, 0))
    new_img.save(output_path)

# Apply Gaussian blur
def apply_gaussian_blur(image, kernel_size=(15, 15)) -> np.ndarray:
    blurred_image = cv2.GaussianBlur(image, kernel_size, 30)
    return blurred_image

#Re-sizing the images
def resize_image(image, size):
    return image.resize(size)

#Re-sizing the image while maintaining its aspect ratio
#!pip install --upgrade Pillow
def resize_image_with_aspect_ratio(image, size):
    # Calculate the aspect ratio
    original_width, original_height = image.size
    aspect_ratio = original_width / original_height

    # Determine whether to resize based on width or height
    new_width, new_height = size
    if new_width is not None and new_height is None:
        new_height = int(new_width / aspect_ratio)
    elif new_height is not None and new_width is None:
        new_width = int(new_height * aspect_ratio)

    # Resize the image
    resized_image = image.resize((new_width, new_height), Image.LANCZOS )

    return resized_image


#Wrapper function to perform several operations on the image.
def wrapper_process_image(image, operations):
    output = image.copy() 
    for operation in operations:
        func = operation[0]
        args = operation[1:]
        output = func(output, *args)
    return output

#Overlaying the image on the template
def overlay_image_on_layout(layout,img, origin, size):
    # Load and resize the image for the panel
    #img = Image.open(image_path)
    img = resize_image_with_aspect_ratio(img, size)
    layout.paste(img, origin)
    return layout

#Function to insert the image into the panels within the layout
def insert_images_into_panels(layout, image_paths, panel_info):
    for panel, img_path in zip(panel_info.keys(), image_paths):
        origin = panel_info[panel]['origin']
        size = panel_info[panel]['size']
        layout = overlay_image_on_layout(layout, img_path, origin, size)
    return layout

#Generating the template for each page
def generate_custom_layout(rows):
    width=1200
    height = 1800
    border_thickness = 4
    background_image = Image.new('RGB', (width,height), color = 'white')
    draw = ImageDraw.Draw(background_image)
    
    row_height = height // len(rows)
    
    y1 = 0
    panel_details = {}
    panel_id = 1
    for row in rows:
        if row == 3:
            third_width = width // 3
            x1 = 0
            for _ in range(3):
                x2 = x1 + third_width
                draw.rectangle([(x1, y1), (x2, y1 + row_height)], outline='black', width=border_thickness)
                panel_details[f'panel_{panel_id}'] = {'origin': (x1, y1), 'size': (x2-x1, row_height)}
                panel_id += 1
                x1 = x2
        elif row == 2:
            half_width = width // 2
            x1 = 0
            for _ in range(2):
                x2 = x1 + half_width
                draw.rectangle([(x1, y1), (x2, y1 + row_height)], outline='black', width=border_thickness)
                panel_details[f'panel_{panel_id}'] = {'origin': (x1, y1), 'size': (x2-x1, row_height)}
                panel_id += 1
                x1 = x2
        elif row == 4:
            quarter_width = width // 4
            x1 = 0
            for _ in range(4):
                x2 = x1 + quarter_width
                draw.rectangle([(x1, y1), (x2, y1 + row_height)], outline='black', width=border_thickness)
                panel_details[f'panel_{panel_id}'] = {'origin': (x1, y1), 'size': (x2-x1, row_height)}
                panel_id += 1
                x1 = x2
        y1 += row_height
        
    return background_image, panel_details

In [32]:
#Need to merge a coulple of images into scene5. This is a one time run so leave it commented
"""
for i in range(10,14):
    combine_images_horizontally(‘images/s5-‘+str(i)+‘.jpg’,‘images/single_img.jpg’,‘images/s5-‘+str(i)+‘.jpg’)
    """

'\nfor i in range(10,14):\n    combine_images_horizontally(‘images/s5-‘+str(i)+‘.jpg’,‘images/single_img.jpg’,‘images/s5-‘+str(i)+‘.jpg’)\n    '

In [33]:
import os
import re
import itertools
import cv2

def extract_image_number_key(image_path):
    numbers = re.findall(r'(\d+)', image_path)
    if numbers:
        return int(numbers[0])
    return 0

def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower()
            for text in re.split('([0-9]+)', s)]

def reading_images(file_path,start_string):
    image_files = [f for f in os.listdir(file_path)
               if f.startswith(start_string) and f.lower().endswith((".jpg",".png",".jpeg", ".JPG", ".PNG", ".JPEG"))]
    image_files = sorted(image_files, key = natural_sort_key)
    return image_files

def get_combined_image_list(modified_images_path, sub_images_path, start_string):
    # Fetch images from the modified_images directory using reading_images function
    modified_images_list = reading_images(modified_images_path, start_string)
    
    # List all filenames in the sub_images directory
    sub_images_list = reading_images(sub_images_path, start_string)
    # Replace images in modified_images_list with those from sub_images if they exist
    final_images_list = []
    for img in modified_images_list:
        if img in sub_images_list:
            final_images_list.append(os.path.join(sub_images_path, img))
        else:
            final_images_list.append(os.path.join(modified_images_path, img))

    return sorted(final_images_list, key = extract_image_number_key)

#subs_path = r"subtitles/english.csv"
subs_path = r"subtitles/chineese.csv"
dialogues = dialogueMap(subs_path)    
#print(dialogues)

In [34]:
#Generalized function to handle all of the scenes
def process_scene(scene_config, page1, page2, darken_image_list, brighten_image_list, blur_image_list, border_image_list):
    
    global image_format
    input_path = r'images/'
    modified_path = r"modified_images/"
    sub_images = r"sub_images/"
    
    image_files = reading_images(input_path, scene_config)
    
    for i in image_files:
        operation = []
        if i in darken_image_list: 
            operation.append((darken_image, 30))
        if i in brighten_image_list:
            operation.append((brighten_image, 20))
        if i in blur_image_list:
            operation.append((apply_gaussian_blur,))
        if i in border_image_list:
            operation.append((add_border_to_image,))
        
        dialogue = getDialouge(dialogues, i.split('.')[0])
        
        # Check if image belongs to scene 5
        if scene_config == 's5-':
            
            #Select the images that go into flashback mode here
            flashback_images = [str(x) + y for x in [scene_config + str(i) for i in range(3, 25)] for y in image_format]
            
            if i in flashback_images:
                if os.path.isfile(input_path+str(i)):
                    img = process_image(input_path+str(i))
                    img.save(modified_path+str(i))
                elif os.path.isfile(sub_images+str(i)):
                    img = process_image(sub_images+str(i))
                    img.save(modified_path+str(i))
                
                # If dialogue exists, add to the image post processing
                if dialogue:
                    add_dialogue_to_image_bottom(modified_path+str(i), dialogue, sub_images+str(i))
                
                # Continue to the next image as we've processed this one
                continue
        
        if os.path.isfile(sub_images + str(i)):
            image = cv2.imread(sub_images+str(i))
        else:
            image = cv2.imread(input_path+str(i))
        
        processed_image = wrapper_process_image(image, operation) 
        cv2.imwrite(modified_path+str(i), processed_image)
        
        if dialogue and not os.path.isfile(sub_images+str(i)):
            add_dialogue_to_image_bottom(modified_path+str(i), dialogue, sub_images+str(i))
    
    layout1, frame1 = generate_custom_layout(page1)
    layout2, frame2 = generate_custom_layout(page2)
    
    modified_image_files = get_combined_image_list(modified_path, sub_images, scene_config)
    #print(modified_image_files)
    
    #Overwritting with the subtitle generated file
    #modified_image_files = 
    
    for i, image_path in enumerate(modified_image_files):
        
        if 1 <= i + 1 <= sum(page1):  
            panel_key = 'panel_' + str(i + 1)
            layout1 = overlay_image_on_layout(layout1, Image.open(image_path), frame1[panel_key]['origin'], frame1[panel_key]['size'])
        elif sum(page1) < i + 1 <= sum(page1) + sum(page2):  
            panel_key = 'panel_' + str(i + 1 - (sum(page1)))
            layout2 = overlay_image_on_layout(layout2, Image.open(image_path), frame2[panel_key]['origin'], frame2[panel_key]['size'])

    return layout1, layout2


In [ ]:
image_format = [".jpg",".png",".jpeg", ".JPG", ".PNG", ".JPEG"]
layouts = []

#calling for scene1
page1 = [2,3,3,3]
page2 = [3,3,3,2]
sceneConfig = 's1-'
darken_image_list = [str(x) + y for x in [sceneConfig + str(i) for i in range(1, 11)] for y in image_format]
brighten_image_list = [str(x) + y for x in [sceneConfig + str(i) for i in [12, 13,15,17]] for y in image_format]
blur_image_list = [str(x) + y for x in [sceneConfig+ str(i) for i in [14, 15]] for y in image_format]
border_image_list = [str(x) + y for x in [sceneConfig + str(i) for i in range(1, 23)] for y in image_format]

layout1,layout2 = process_scene(sceneConfig,page1,page2,darken_image_list,brighten_image_list,blur_image_list,border_image_list)
layouts.append(layout1)
layouts.append(layout2)

#calling for scene2
page3 = [3,3,3]
page4 = [3,3,3]
sceneConfig = "s2-"
darken_image_list = [str(x) + y for x in [sceneConfig+ str(i) for i in [2,5,7,11,13,16,17]] for y in image_format]
brighten_image_list = [str(x) + y for x in [sceneConfig+ str(i) for i in [4,6,8,10,15]] for y in image_format]
blur_image_list = [str(x) + y for x in [sceneConfig+ str(i) for i in [2,6,7]] for y in image_format]
border_image_list = [str(x) + y for x in [sceneConfig+ str(i) for i in range(1, 19)] for y in image_format]

layout3,layout4 = process_scene(sceneConfig,page3,page4,darken_image_list,brighten_image_list,blur_image_list,border_image_list)
layouts.append(layout3)
layouts.append(layout4)

#calling for scene3
page5 = [2,3,3,2]
page6 = [3,3,2,3]
sceneConfig = "s3-"
darken_image_list = [str(x) + y for x in [sceneConfig+ str(i) for i in [2,8,11,13,15,16,18]] for y in image_format]
brighten_image_list = [str(x) + y for x in [sceneConfig+ str(i) for i in [4,5,6,7]] for y in image_format]
blur_image_list = [str(x) + y for x in [sceneConfig+ str(i) for i in [2,6,7,9,13]] for y in image_format]
border_image_list = [str(x) + y for x in [sceneConfig+ str(i) for i in range(1, 22)] for y in image_format]

layout5,layout6 = process_scene(sceneConfig,page5,page6,darken_image_list,brighten_image_list,blur_image_list,border_image_list)
layouts.append(layout5)
layouts.append(layout6)

#calling for scene4
page7 = [3,2,2,3]
page8 = [3,2,2,3]
sceneConfig = "s4-"
darken_image_list = [str(x) + y for x in [sceneConfig + str(i) for i in [5, 10, 14]] for y in image_format]
brighten_image_list = [str(x) + y for x in [sceneConfig + str(i) for i in [7, 8, 11, 12, 13, 15, 16, 17]] for y in image_format]
blur_image_list = [str(x) + y for x in [sceneConfig + str(i) for i in [9,10,11,12,13]] for y in image_format]
border_image_list = [str(x) + y for x in [sceneConfig + str(i) for i in range(1, 21)] for y in image_format]

layout7,layout8 = process_scene(sceneConfig,page7,page8,darken_image_list,brighten_image_list,blur_image_list,border_image_list)
layouts.append(layout7)
layouts.append(layout8)


#calling for scene5
page9 = [3,3,3,3]
page10 = [3,3,3,3]
sceneConfig = 's5-'
darken_image_list = [str(x) + y for x in [sceneConfig+ str(i) for i in range(1,11)] for y in image_format]
brighten_image_list = [str(x) + y for x in [sceneConfig+ str(i) for i in [12, 13,14,15,16,17,18]] for y in image_format]
blur_image_list = [str(x) + y for x in [sceneConfig+ str(i) for i in [14, 15,16,17]] for y in image_format]
border_image_list = [str(x) + y for x in [sceneConfig+ str(i) for i in range(1, 25)] for y in image_format]

layout9,layout10 = process_scene(sceneConfig,page9,page10,darken_image_list,brighten_image_list,blur_image_list,border_image_list)
layouts.append(layout9)
layouts.append(layout10)



C:\Users\cheth\AppData\Local\Temp\ipykernel_11624\3742065936.py:36: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  if font.getsize(line)[0] <= max_width:
C:\Users\cheth\AppData\Local\Temp\ipykernel_11624\3742065936.py:44: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  if font.getsize(new_line + (' ' if language == 'english' else '') + word)[0] <= max_width:
C:\Users\cheth\AppData\Local\Temp\ipykernel_11624\3742065936.py:82: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  w, h = draw.textsize(text, font=font)
C:\Users\cheth\AppData\Local\Temp\ipykernel_11624\3742065936.py:36: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  if font.getsize(line)[0] <= max_width:
C:\Users\cheth\App

C:\Users\cheth\AppData\Local\Temp\ipykernel_11624\3742065936.py:36: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  if font.getsize(line)[0] <= max_width:
C:\Users\cheth\AppData\Local\Temp\ipykernel_11624\3742065936.py:82: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  w, h = draw.textsize(text, font=font)
C:\Users\cheth\AppData\Local\Temp\ipykernel_11624\3742065936.py:36: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  if font.getsize(line)[0] <= max_width:
C:\Users\cheth\AppData\Local\Temp\ipykernel_11624\3742065936.py:44: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  if font.getsize(new_line + (' ' if language == 'english' else '') + word)[0] <= max_width:
C:\Users\cheth\App

C:\Users\cheth\AppData\Local\Temp\ipykernel_11624\3742065936.py:36: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  if font.getsize(line)[0] <= max_width:
C:\Users\cheth\AppData\Local\Temp\ipykernel_11624\3742065936.py:44: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  if font.getsize(new_line + (' ' if language == 'english' else '') + word)[0] <= max_width:
C:\Users\cheth\AppData\Local\Temp\ipykernel_11624\3742065936.py:82: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  w, h = draw.textsize(text, font=font)
C:\Users\cheth\AppData\Local\Temp\ipykernel_11624\3742065936.py:36: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  if font.getsize(line)[0] <= max_width:
C:\Users\cheth\App

C:\Users\cheth\AppData\Local\Temp\ipykernel_11624\3742065936.py:36: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  if font.getsize(line)[0] <= max_width:
C:\Users\cheth\AppData\Local\Temp\ipykernel_11624\3742065936.py:82: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  w, h = draw.textsize(text, font=font)
C:\Users\cheth\AppData\Local\Temp\ipykernel_11624\3742065936.py:36: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  if font.getsize(line)[0] <= max_width:
C:\Users\cheth\AppData\Local\Temp\ipykernel_11624\3742065936.py:44: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  if font.getsize(new_line + (' ' if language == 'english' else '') + word)[0] <= max_width:
C:\Users\cheth\App

C:\Users\cheth\AppData\Local\Temp\ipykernel_11624\3742065936.py:36: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  if font.getsize(line)[0] <= max_width:
C:\Users\cheth\AppData\Local\Temp\ipykernel_11624\3742065936.py:82: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  w, h = draw.textsize(text, font=font)
C:\Users\cheth\AppData\Local\Temp\ipykernel_11624\3742065936.py:36: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  if font.getsize(line)[0] <= max_width:
C:\Users\cheth\AppData\Local\Temp\ipykernel_11624\3742065936.py:82: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  w, h = draw.textsize(text, font=font)
C:\Users\cheth\AppData\Local\Temp\ipykernel_11624\3742065936.py:36: 

In [ ]:
#To save it as a PDF

def save_layouts_as_pdf(layouts, filename):
    
    # Convert all layouts to the same mode (necessary for saving as PDF)
    layouts = [layout.convert('RGB') for layout in layouts]
    layouts[0].save(filename, save_all=True, append_images=layouts[1:])
    
#save_layouts_as_pdf(layouts, 'output-english.pdf')
save_layouts_as_pdf(layouts, 'output-chineese.pdf')

